In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace 'your_file_path' with the actual path to your CSV file in Google Drive
file_path = '/data_file_path'

try:
  df = pd.read_csv(file_path)
except FileNotFoundError:
  print(f"File not found at: {file_path}")

In [ ]:
df.head(1)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1902147046548889693,Tue Mar 18 23:56:10 +0000 2025,0,Pada 18 Maret 2025 kemarin IHSG anjlok lebih d...,1902147046548889693,https://pbs.twimg.com/media/GmXIp9Pa8AAWcP2.jpg,NaN,in,"Jakarta Utara, DKI Jakarta",0,1,0,https://x.com/gunawansanja/status/190214704654...,428174607,gunawansanja


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
#@title Installing Groq
!pip install groq -q
from google.colab import userdata
from groq import Groq
client = Groq(
    api_key=userdata.get('groqapi'),
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.9 MB/s eta 0:00:00


In [ ]:
user_prompt = """\
The text below is an Indonesia Twitter/X. \
The text is about IHSG stock, index of all stocks listed on the Indonesia Stock Exchange \
\
You have three main task: \
1. Classify the text into Neutral, Negative, or Positive. \
2. Classify the text into 8 Plutchik's emotions: Joy, Trust, Fear, Surprise, Sadness, Disgust, Anger, Anticipation. \
3. Summarize the text in one sentence. \
\
Ignore user mentions in the text. \
Ignore links in the text. \
\
Format the output to one word: Neutral, Negative, or Positive | \
Format the output to one word: Joy, Trust, Fear, Surprise, Sadness, Disgust, Anger, Anticipation | \
Format the output to one sentence in a simple BAHASA INDONESIA, NO LONGER THAN 10 WORDS. \
\
ENSURE THAT THE RESPONSE IS IN THE FORMAT ABOVE. \
No need to add anything else. \
\
Example: Negative | Anger | Saya sangat senang hari ini. \
\
Text:
"""

In [ ]:
#@title Completion Method
import re
import time

def completion(user_prompt, text):

    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {
                "role": "user",
                "content": user_prompt + text,
            },
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        stream=False,
        stop=None,
    )
    response = completion.choices[0].message.content
    cleaned_response = re.sub(r"<think>.*?</think>\s*", "", response, flags=re.DOTALL)
    print(cleaned_response)
    # time.sleep(3)
    return cleaned_response
    # sentiment, emotion, summary = cleaned_response.split(' | ')
    # return sentiment, emotion, summary

In [ ]:
text = df.full_text[5]
text

'@Jurigjarian75 @hnirankara Sejarah mana turunya harga ihsg menjatuhkan pemerintahan kan itu ihsg..masih ada saham saham yang positif..tahu nggak konsep ihsg...sok ahli lu...dasar gen z eh milenial'

In [ ]:
response = completion(user_prompt, text)
sentiment, emotion, summary = response.split(' | ')

Negative | Anger | Kritik terhadap pemahaman seseorang tentang IHSG.


In [ ]:
print(sentiment, emotion, summary)

Negative Anger Kritik terhadap pemahaman seseorang tentang IHSG.


In [ ]:
import time

retry_delay = 300  # 5-minute delay
max_retries = 3  # Maximum retry attempts

for index, row in df.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            print(f"Processing index {index} (Attempt {retries + 1}/{max_retries})")
            # if index < 1000:
            #     continue
            text = row['full_text']
            response = completion(user_prompt, text)
            sentiment, emotion, summary = response.split(' | ')
            df.loc[index, 'sentiment'] = sentiment
            df.loc[index, 'emotion'] = emotion
            df.loc[index, 'summary'] = summary
            break
        except Exception as e:
            retries += 1
            print(f"Error at index {index}: {e}")

            if retries < max_retries:
                print(f"Retrying in {retry_delay} seconds... ({retries}/{max_retries})")
                time.sleep(retry_delay)  # Wait before retrying
            else:
                print(f"Max retries reached for index {index}. Skipping...")

Processing index 0 (Attempt 1/3)
0
Negative | Fear | IHSG anjlok lebih dari 5% pada 18 Maret 2025.
Processing index 1 (Attempt 1/3)
1
Negative | Anticipation | Masyarakat khawatir IHSG anjlok akibat korupsi.
Processing index 2 (Attempt 1/3)
2
Negative | Surprise | IHSG turun 21,7%.
Processing index 3 (Attempt 1/3)
3
Negative | Anger | Pemilih 02 tidak mengerti IHSG dan rupiah.
Processing index 4 (Attempt 1/3)
4
Negative | Anger | Daya beli menurun dan IHSG anjlok.
Processing index 5 (Attempt 1/3)
5
Negative | Anger | Kritik orang karena tidak mengerti konsep IHSG.
Processing index 6 (Attempt 1/3)
6
Positive | Trust | IHSG: Keputusan suku bunga BI diharapkan meredakan pasar.
Processing index 7 (Attempt 1/3)
7
Negative | Fear | IHSG mengalami penurunan signifikan.
Processing index 8 (Attempt 1/3)
8
Negative | Fear | IHSG anjlok hingga trading halt, BEI imbau investor cermat.
Processing index 9 (Attempt 1/3)
9
Neutral | Trust | IHSG itu apa kak?
Processing index 10 (Attempt 1/3)
10
Negati

KeyboardInterrupt: 

In [ ]:
# prompt: save and download the df

# Save the DataFrame to a CSV file in Google Drive
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment1.csv', index=False)

# Download the CSV file to your local machine
from google.colab import files
files.download('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment1.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Continue Labeling


In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment1.csv'

a = pd.read_csv(file_path)

In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
for index, row in a.iterrows():
    if index < 954:
        continue
    if index > 1000:
        break
    print(f"Processing index {index}")
    text = row['full_text']
    response = completion(user_prompt, text)
    sentiment, emotion, summary = response.split(' | ')
    a.loc[index, 'sentiment'] = sentiment
    a.loc[index, 'emotion'] = emotion
    a.loc[index, 'summary'] = summary


Processing index 954
Positive | Joy | IHSG mengalami kenaikan hari ini.
Processing index 955
Negative | Surprise | IHSG halt gara-gara saham.
Processing index 956
Negative | Disgust | Akun tanpa pengetahuan atau saham jadi ahli saham.
Processing index 957
Negative | Sadness | IHSG anjlok 5%, BUMN diberi tahan saat kapal tenggelam.
Processing index 958
Negative | Fear | IHSG ambruk lagi, membuat was-was.
Processing index 959
Negative | Fear | IHSG dikuasai asing, khawatir rupiah dibuang.
Processing index 960
Neutral | Trust | Pengguna membagikan tautan tentang IHSG.
Processing index 961
Negative | Fear | Tangan asing menyebabkan IHSG turun dan menjatuhkan pemerintah hari ini.
Processing index 962
Positive | Trust | IHSG turun dianggap akal-akalan asing, tetap oke.
Processing index 963
Negative | Fear | IHSG turun dan dua kejadian besar terjadi dalam sebulan.
Processing index 964
Negative | Anger | IHSG merosot, siapa yang bersalah?
Processing index 965
Negative | Disgust | IHSG jatuh, p

In [ ]:
# Save the DataFrame to a CSV file in Google Drive
a.to_csv('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment3.csv', index=False)

# Download the CSV file to your local machine
from google.colab import files
files.download('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
a.info() #2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
a.info() #3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment4.csv'

dfc = pd.read_csv(file_path)
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
import time
from google.colab import files


for index, row in dfc.iterrows():
      if index < 2244:
          continue
      try:
          print(f"Processing index {index}")
          text = row['full_text']
          response = completion(user_prompt, text)
          sentiment, emotion, summary = response.split(' | ')
          dfc.loc[index, 'sentiment'] = sentiment
          dfc.loc[index, 'emotion'] = emotion
          dfc.loc[index, 'summary'] = summary
      except Exception as e:
          print(f"Error at index {index}: {e}")
          dfc.to_csv('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment4.csv', index=False)
          files.download('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_sentiment4.csv')
          print(dfc.info())
          break

Processing index 2244
Negative | Fear | IHSG ditutup turun 3,84% setelah trading halt.
Processing index 2245
Negative | Fear | IHSG hancur hingga perdagangan di-suspend.
Processing index 2246
Negative | Anger | Pemerintah menyebabkan crash IHSG dengan kebijakan negatifnya.
Processing index 2247
Negative | Fear | Orang Indonesia lebih memilih saham AS karena IHSG tidak stabil.
Processing index 2248
Positive | Anticipation | Hipotesa Dr. Bahlul tentang label hemat energi akan tingkatkan IHSG.
Processing index 2249
Negative | Anticipation | IHSG turun, semua aset dipindahkan ke luar negeri.
Processing index 2250
Positive | Anticipation | Yuk IHSG bisa naik.
Processing index 2251
Negative | Fear | Ragu untuk beli SBN karena IHSG dan utang yang banyak.
Processing index 2252
Positive | Trust | IHSG naik 2%, hoax trading halt dibantah.
Processing index 2253
Neutral | Fear | Beberapa faktor yang mempengaruhi IHSG.
Processing index 2254
Negative | Fear | IHSG hanya turun saat krisis dan pandemi

In [ ]:
dfc.to_csv('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final.csv', index=False)
files.download('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

# Huggingface Labelling

In [ ]:
#@title Read DF
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final.csv'

df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      2738 non-null   int64 
 1   created_at               2738 non-null   object
 2   favorite_count           2738 non-null   int64 
 3   full_text                2738 non-null   object
 4   id_str                   2738 non-null   int64 
 5   image_url                332 non-null    object
 6   in_reply_to_screen_name  1364 non-null   object
 7   lang                     2738 non-null   object
 8   location                 1494 non-null   object
 9   quote_count              2738 non-null   int64 
 10  reply_count              2738 non-null   int64 
 11  retweet_count            2738 non-null   int64 
 12  tweet_url                2738 non-null   object
 13  user_id_str              2738 non-null   int64 
 14  username                 2738 non-null  

In [ ]:
!pip install transformers torch -q

import pandas as pd
from transformers import pipeline

# Load the pre-trained model
classifier = pipeline("sentiment-analysis", model="Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis")

# Function to get sentiment
def get_sentiment(text):
    try:
        result = classifier(text)[0]
        return result['label']
    except Exception as e:
        print(f"Error processing text: {text}, Error: {e}")
        return "Error"  # or handle the error as needed

# Apply the function to the 'full_text' column and create a new 'sentiment' column
df['sentiment_hf'] = df['full_text'].apply(get_sentiment)

# Display the updated df
df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00


model.safetensors:  78%|#######8  | 346M/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,sentiment,emotion,summary,sentiment_hf
0,1902147046548889693,Tue Mar 18 23:56:10 +0000 2025,0,Pada 18 Maret 2025 kemarin IHSG anjlok lebih d...,1902147046548889693,https://pbs.twimg.com/media/GmXIp9Pa8AAWcP2.jpg,NaN,in,"Jakarta Utara, DKI Jakarta",0,1,0,https://x.com/gunawansanja/status/190214704654...,428174607,gunawansanja,Negative,Fear,IHSG anjlok lebih dari 5% pada 18 Maret 2025.,Neutral
1,1902147041725493528,Tue Mar 18 23:56:09 +0000 2025,831,Melihat berita korupsi hampir tiap saat daya b...,1902147041725493528,https://pbs.twimg.com/ext_tw_video_thumb/19021...,NaN,in,Indonesia,7,37,328,https://x.com/Hilmi28/status/1902147041725493528,65067181,Hilmi28,Negative,Anticipation,Masyarakat khawatir IHSG anjlok akibat korupsi.,Negative
2,1902146999379726720,Tue Mar 18 23:55:59 +0000 2025,0,Gila IHSG turun 21 7%?,1902146999379726720,NaN,NaN,in,NaN,0,1,0,https://x.com/hazelviewpoint/status/1902146999...,1606266207945965568,hazelviewpoint,Negative,Surprise,"IHSG turun 21,7%.",Negative
3,1902146849303359802,Tue Mar 18 23:55:23 +0000 2025,0,But 02 voters stay dumb. Still defending wowo ...,1902146849303359802,NaN,NaN,en,".・゜゜・I'm indeed a Jay solostan, BUT I AIN'T NO...",0,0,0,https://x.com/jaycatyong/status/19021468493033...,1365312980720656385,jaycatyong,Negative,Anger,Pemilih 02 tidak mengerti IHSG dan rupiah.,Positive
4,1902146788402094387,Tue Mar 18 23:55:09 +0000 2025,779,tahun ini udah daya beli menurun trus IHSG an...,1902146788402094387,https://pbs.twimg.com/media/GmXIbF7XwAANqop.jpg,NaN,in,Indonesia,9,12,187,https://x.com/tanyarlfes/status/19021467884020...,1371650588,tanyarlfes,Negative,Anger,Daya beli menurun dan IHSG anjlok.,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733,1901900034138095779,Tue Mar 18 09:16:51 +0000 2025,0,@profesor_saham Emang bisa ya kedatangan DPR m...,1901925756512432526,NaN,profesor_saham,in,NaN,0,0,0,https://x.com/HidyNoora/status/190192575651243...,1760523689366630400,HidyNoora,Neutral,Anticipation,Pertanyaan tentang pengaruh DPR terhadap IHSG.,Neutral
2734,1901925706403135937,Tue Mar 18 09:16:39 +0000 2025,0,Beli durian di Kota Medan Main di Moli77 hoki ...,1901925706403135937,https://pbs.twimg.com/ext_tw_video_thumb/19019...,NaN,in,NaN,0,0,0,https://x.com/bigmone_Q/status/190192570640313...,158372921,bigmone_Q,Positive,Anticipation,Beli durian di Medan dan main di Moli77.,Neutral
2735,1901870798077595898,Tue Mar 18 09:16:18 +0000 2025,0,@avadaakedavrae kalo liat ihsg sbnrnya asal pe...,1901925621367971971,NaN,avadaakedavrae,in,Padang Mahsyar,0,1,0,https://x.com/txtfrompiranha/status/1901925621...,1277234915168227328,txtfrompiranha,Negative,Fear,Kebijakan pemerintah yang tidak baik mengusir ...,Negative
2736,1901886677049626829,Tue Mar 18 09:15:49 +0000 2025,0,@ardisatriawan Jadi bursa mengalami penurunan ...,1901925496440655912,NaN,ardisatriawan,in,NaN,0,0,0,https://x.com/ffrhmnnnn/status/190192549644065...,1506085233451421699,ffrhmnnnn,Negative,Fear,IHSG turun 5% di sesi bursa pagi hari ini.,Negative


In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = 0  # Use the first available GPU
else:
    device = -1  # Use CPU if GPU is not available

# Load the pre-trained model with specified device
classifier = pipeline("text-classification", model="Aardiiiiy/NusaBERT-base-Indonesian-Plutchik-emotion-analysis-v2", device=device)

# Function to get emotion
def get_emotion(text):
    try:
        result = classifier(text)[0]
        return result['label']
    except Exception as e:
        print(f"Error processing text: {text}, Error: {e}")
        return "Error"  # or handle the error as needed

# Apply the function to the 'clean_text' column and create a new 'emotion' column
df['emotion_hf'] = df['full_text'].apply(get_emotion)

# Display the updated df
df


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/261k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,sentiment,emotion,summary,sentiment_hf,emotion_hf
0,1902147046548889693,Tue Mar 18 23:56:10 +0000 2025,0,Pada 18 Maret 2025 kemarin IHSG anjlok lebih d...,1902147046548889693,https://pbs.twimg.com/media/GmXIp9Pa8AAWcP2.jpg,NaN,in,"Jakarta Utara, DKI Jakarta",0,1,0,https://x.com/gunawansanja/status/190214704654...,428174607,gunawansanja,Negative,Fear,IHSG anjlok lebih dari 5% pada 18 Maret 2025.,Neutral,Surprise
1,1902147041725493528,Tue Mar 18 23:56:09 +0000 2025,831,Melihat berita korupsi hampir tiap saat daya b...,1902147041725493528,https://pbs.twimg.com/ext_tw_video_thumb/19021...,NaN,in,Indonesia,7,37,328,https://x.com/Hilmi28/status/1902147041725493528,65067181,Hilmi28,Negative,Anticipation,Masyarakat khawatir IHSG anjlok akibat korupsi.,Negative,Sadness
2,1902146999379726720,Tue Mar 18 23:55:59 +0000 2025,0,Gila IHSG turun 21 7%?,1902146999379726720,NaN,NaN,in,NaN,0,1,0,https://x.com/hazelviewpoint/status/1902146999...,1606266207945965568,hazelviewpoint,Negative,Surprise,"IHSG turun 21,7%.",Negative,Surprise
3,1902146849303359802,Tue Mar 18 23:55:23 +0000 2025,0,But 02 voters stay dumb. Still defending wowo ...,1902146849303359802,NaN,NaN,en,".・゜゜・I'm indeed a Jay solostan, BUT I AIN'T NO...",0,0,0,https://x.com/jaycatyong/status/19021468493033...,1365312980720656385,jaycatyong,Negative,Anger,Pemilih 02 tidak mengerti IHSG dan rupiah.,Positive,Disgust
4,1902146788402094387,Tue Mar 18 23:55:09 +0000 2025,779,tahun ini udah daya beli menurun trus IHSG an...,1902146788402094387,https://pbs.twimg.com/media/GmXIbF7XwAANqop.jpg,NaN,in,Indonesia,9,12,187,https://x.com/tanyarlfes/status/19021467884020...,1371650588,tanyarlfes,Negative,Anger,Daya beli menurun dan IHSG anjlok.,Negative,Disgust
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733,1901900034138095779,Tue Mar 18 09:16:51 +0000 2025,0,@profesor_saham Emang bisa ya kedatangan DPR m...,1901925756512432526,NaN,profesor_saham,in,NaN,0,0,0,https://x.com/HidyNoora/status/190192575651243...,1760523689366630400,HidyNoora,Neutral,Anticipation,Pertanyaan tentang pengaruh DPR terhadap IHSG.,Neutral,Surprise
2734,1901925706403135937,Tue Mar 18 09:16:39 +0000 2025,0,Beli durian di Kota Medan Main di Moli77 hoki ...,1901925706403135937,https://pbs.twimg.com/ext_tw_video_thumb/19019...,NaN,in,NaN,0,0,0,https://x.com/bigmone_Q/status/190192570640313...,158372921,bigmone_Q,Positive,Anticipation,Beli durian di Medan dan main di Moli77.,Neutral,Joy
2735,1901870798077595898,Tue Mar 18 09:16:18 +0000 2025,0,@avadaakedavrae kalo liat ihsg sbnrnya asal pe...,1901925621367971971,NaN,avadaakedavrae,in,Padang Mahsyar,0,1,0,https://x.com/txtfrompiranha/status/1901925621...,1277234915168227328,txtfrompiranha,Negative,Fear,Kebijakan pemerintah yang tidak baik mengusir ...,Negative,Trust
2736,1901886677049626829,Tue Mar 18 09:15:49 +0000 2025,0,@ardisatriawan Jadi bursa mengalami penurunan ...,1901925496440655912,NaN,ardisatriawan,in,NaN,0,0,0,https://x.com/ffrhmnnnn/status/190192549644065...,1506085233451421699,ffrhmnnnn,Negative,Fear,IHSG turun 5% di sesi bursa pagi hari ini.,Negative,Sadness


In [ ]:
# Save the DataFrame to a CSV file in Google Drive
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final_adahf.csv', index=False)

# Download the CSV file to your local machine
from google.colab import files
files.download('/content/drive/MyDrive/Colab Notebooks/Proyek Analisis Data - Sospol/IHSG 18 Maret 2025/ihsg_saja_latest_final_adahf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>